# Tutorial 2: Data access advanced concepts
### `themachinethatgoesping` tutorial series

`themachinethatgoesping` concepts covered:
- Advanced use of find_files()
- Advanced use of File Handler
- Advanced access of Pings

In [1]:
%matplotlib widget

import numpy as np
import themachinethatgoesping as theping
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

### 1. Advanced use of find_files()

In [2]:
# find_files() searches the input folder recursively (aka, including subfolders)
folder = '../unittest_data'
files = theping.echosounders.index_functions.find_files(folder, [".all","wcd"])
print(len(files))

Found 18 files
18


In [3]:
# find_files() can use a list of folders as input
folders = []
folders.append('../unittest_data/kongsberg/simon/')
folders.append('../unittest_data/kongsberg/turbeams/')
files = theping.echosounders.index_functions.find_files(folders, [".all","wcd"])
print(len(files))

Found 6 files
6


In [4]:
# pairs of files (e.g. .all and .wcd) don't have to be in the same folders. 
# Remember they are only paired later, by a File Handler

### 2. Advanced use of File Handler

In [5]:
# Create a File Handler to access the raw data files
# TO DO - modify to talk about caching
folder = '../unittest_data'
files = theping.echosounders.index_functions.find_files(folder, [".all","wcd"])
fileHandler = theping.echosounders.kongsbergall.KongsbergAllFileHandler(files)

Found 18 files
indexing files ⠐ 100% :00s<00m:00s] [..0057642499202963.wcd (1/18)]                               
indexing files ⠠ 100% :00s<00m:00s] [..4075547798063212.all (18/18)]                                
indexing files ⢀ 100% :00s<00m:00s] [Found: 729 datagrams in 18 files (10MB)]                                         
Initializing ping interface ⢀ 90% :00s<00m:00s] [Done]                                              


### 3. Advanced access of Pings

In [6]:
# pingcontainer are mostly for fast filtering of pings (eg on basis of sonar head, or location)

In [7]:
# Access an individual ping in a Ping Container by indexing
pingContainer = fileHandler.get_pings()
ping42 = pingContainer[42]
type(ping42)

themachinethatgoesping.echosounders_cppy.kongsbergall.filetypes.KongsbergAllPing

In [8]:
#  This approach only works for a single ping. If you index a Ping Container for multiple pings, you will get a (smaller) Ping Container object
pings0To42 = pingContainer[0:42]
type(pings0To42)

themachinethatgoesping.echosounders_cppy.kongsbergall.filedatacontainers.KongsbergAllPingContainer

In [9]:
# If you want a list of Ping objects, you need to loop over each element of the Ping Container
pings0To42 = []
for i in range(42):
    pings0To42.append(pingContainer[i])
print(f"pings0To42 is now a {type(pings0To42)} where each element is a {type(pings0To42[0])}")

pings0To42 is now a <class 'list'> where each element is a <class 'themachinethatgoesping.echosounders_cppy.kongsbergall.filetypes.KongsbergAllPing'>


In [10]:
# But you can also directly create lists of pings selected from a Ping Container by filtering for a specific critiera, for example pings containing watercolumn data
pingsWithWC = theping.pingprocessing.filter_pings.by_features(pingContainer,['watercolumn.amplitudes'])
print(f"pingsWithWC is a {type(pingsWithWC)} where each element is a {type(pingsWithWC[0])}")

pingsWithWC is a <class 'list'> where each element is a <class 'themachinethatgoesping.echosounders_cppy.kongsbergall.filetypes.KongsbergAllPing'>


### 4. Get datagrams from all files

In [11]:
print(fileHandler.datagram_interface.datagrams())

DatagramContainer
#################
-
Time info (Datagrams) 
------------------------ 
- Start time: 21/08/2012 17:09:42.36 
- End time:   26/07/2024 15:02:51.40 
- Sorted:     no                     

 Contained datagrams 
---------------------       
- Total:                                       729 
- Datagrams [PUIDOutput]:                      3   [30]
- Datagrams [PUStatusOutput]:                  9   [31]
- Datagrams [ExtraParameters]:                 3   [33]
- Datagrams [AttitudeDatagram]:                28  [41]
- Datagrams [ClockDatagram]:                   17  [43]
- Datagrams [SurfaceSoundSpeedDatagram]:       1   [47]
- Datagrams [InstallationParametersStart]:     18  [49]
- Datagrams [RawRangeAndAngle]:                80  [4e]
- Datagrams [QualityFactorDatagram]:           6   [4f]
- Datagrams [PositionDatagram]:                31  [50]
- Datagrams [RuntimeParameters]:               38  [52]
- Datagrams [SoundSpeedProfileDatagram]:       18  [55]
- Datagrams [XYZDatagra

In [12]:
# datagram_interface.datagrams() is an iterator so you can loop over all datagrams with:
for d in tqdm(fileHandler.datagram_interface.datagrams()):
    pass

# or loop over specific datagram types with
for d in tqdm(fileHandler.datagram_interface.datagrams("RuntimeParameters")):
    pass

  0%|          | 0/729 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

In [13]:
# index to access a specific datagram
d  = fileHandler.datagram_interface.datagrams("RuntimeParameters")[0]
print(d)

RuntimeParameters
#################
- bytes:               52       
- stx:                 0x02     
- datagram_identifier: 0x52     [RuntimeParameters]
- model_number:        EM2040   [2040]
- date:                20160413 [YYYYMMDD]
- time_since_midnight: 67234243 [ms]

 date/time 
-----------  
- timestamp: 1460.573e⁶   [s]
- date:      13/04/2016   [MM/DD/YYYY]
- time:      18:40:34.243 [HH:MM:SS]

 datagram content 
------------------             
- ping_counter:                         5371       
- system_serial_number:                 213        
- operator_station_status:              0          
- processing_unit_status:               0          
- bsp_status:                           0          
- sonar_head_or_transceiver_status:     0          
- mode:                                 0b00000001 
- filter_identifier:                    0b01101101 
- minimum_depth:                        1          [m]
- maximum_depth:                        50         [m]
- absorption_coe

In [14]:
# and use "get_" methods to get data of interest from a datagram
print(f"Model number: {d.get_model_number()}")
print(f"Mode: {d.get_mode()}")
print(f"System serial number: {d.get_system_serial_number()}")
print(f"Transmit pulse length (raw): {d.get_transmit_pulse_length()}")
print(f"Transmit pulse length (in sec): {d.get_transmit_pulse_length_in_seconds()}")

Model number: 2040
Mode: 1
System serial number: 213
Transmit pulse length (raw): 600
Transmit pulse length (in sec): 0.0005999999702908099


In [15]:
# Note you can also access runtime parameters applying to a given ping
ping = fileHandler.get_pings()[0]
rp = ping.file_data.get_runtime_parameters()
print(f"Transmit pulse length (in sec) for this ping: {d.get_transmit_pulse_length_in_seconds()}")

Transmit pulse length (in sec) for this ping: 0.0005999999702908099


### 4. Access datagrams in individual files

In [16]:
# you can create filehandlers of individual files
file = files[0]
fileHandler = theping.echosounders.kongsbergall.KongsbergAllFileHandler(file)

# you can reuse cache created for the list of files on an individual file
cache = theping.echosounders.index_functions.get_cache_file_paths(files)
fileHandler = theping.echosounders.kongsbergall.KongsbergAllFileHandler(file, file_cache_paths = cache)

indexing file ⠐ 100% :00s<00m:00s]           
indexing file ⠠ 100% :00s<00m:00s] [Found: 10 datagrams]                     
Initializing ping interface ⠐ 0% :00s<00m:00s] [Done]                                              
indexing file ⠐ 83% :00s<00m:00s] [Found: 10 datagrams]                     
Initializing ping interface ⠐ 0% :00s<00m:00s] [Done]                                              


In [17]:

#if you have a big filehandler for many files, you can still access datagrams file by file
fileHandler = theping.echosounders.kongsbergall.KongsbergAllFileHandler(files, file_cache_paths = cache)

#get first runtime parameters datagram of each file
rps = {}
for datainterface in fileHandler.datagramdata_interface.per_primary_file():
    if len(datainterface.datagrams('RuntimeParameters')) > 0:
        rp = datainterface.datagrams('RuntimeParameters')[0]
        rps[datainterface.get_file_path()] = rp
    else:
        print('No runtime parameters in', datainterface.get_file_path())
    

indexing files ⢀ 98% :00s<00m:00s] [Found: 729 datagrams in 18 files (10MB)]                                         
Initializing ping interface ⢀ 90% :00s<00m:00s] [Done]                                              


In [18]:
for k,v in rps.items():
    print(k, 'sonarmode:',v.get_sonar_mode_enabled())

../unittest_data/kongsberg/simon/-8780057642499202963.all sonarmode: False
../unittest_data/kongsberg/a/y/-7609271985000766481.all sonarmode: False
../unittest_data/kongsberg/a/f/ALL/4590722669055497039.all sonarmode: False
../unittest_data/kongsberg/a/c/2720871266855321420.all sonarmode: False
../unittest_data/kongsberg/e/5556426203143536656.all sonarmode: True
../unittest_data/kongsberg/e/6340811117880500921.all sonarmode: True
../unittest_data/kongsberg/g/4467916373654671126.all sonarmode: False
../unittest_data/kongsberg/turbeams/2353008643552583898.all sonarmode: False
../unittest_data/kongsberg/turbeams/-5459828243338449053.all sonarmode: False
../unittest_data/kongsberg/he/8194075547798063212.all sonarmode: False


In [19]:
for k,v in rps.items():
    print(k, 'sonarmode:', v.get_model_number_as_string(),v.get_mode_as_ping_mode())

../unittest_data/kongsberg/simon/-8780057642499202963.all sonarmode: EM2040 300 kHz
../unittest_data/kongsberg/a/y/-7609271985000766481.all sonarmode: EM2040C 400 kHz
../unittest_data/kongsberg/a/f/ALL/4590722669055497039.all sonarmode: EM710 Shallow
../unittest_data/kongsberg/a/c/2720871266855321420.all sonarmode: EM2040 300 kHz
../unittest_data/kongsberg/e/5556426203143536656.all sonarmode: EM2040 400 kHz
../unittest_data/kongsberg/e/6340811117880500921.all sonarmode: EM2040 400 kHz
../unittest_data/kongsberg/g/4467916373654671126.all sonarmode: EM710 Shallow
../unittest_data/kongsberg/turbeams/2353008643552583898.all sonarmode: EM2040 300 kHz
../unittest_data/kongsberg/turbeams/-5459828243338449053.all sonarmode: EM2040 400 kHz
../unittest_data/kongsberg/he/8194075547798063212.all sonarmode: EM710 Shallow
